<a href="https://colab.research.google.com/github/panghanwu/tibame_project/blob/main/neo4j_import_database_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script includes uploading csv data to Neo4j and query algorithm.

In [ ]:
!pip install py2neo
import py2neo as neo
import pandas as pd
import random

In [ ]:
"""load data"""
# product info
product_df = pd.read_csv('drive/MyDrive/Colab Notebooks/tibame_project/graph_database/pro_data_dv.csv')
# relation matrix
relation_df = pd.read_csv('drive/MyDrive/Colab Notebooks/tibame_project/graph_database/relationship_matrix.csv', index_col=0)

"""Relation matrix preprocessing""" 
# fill None with .0
relation_df = relation_df.fillna(0.0)

psn = relation_df.index  # data list
pn = len(psn)  # data size

# add little fluctuation
def fluctuate(d):
  if d == 2.0 or d == 0.0:
    return d
  else:
    r = round(random.uniform(-0.05,0.05), 3)
    return d + r

# def fluctuate(d):
#   if d == 0:
#     return d + 1

for i in range(pn):
  relation_df[psn[i]] = relation_df[psn[i]].apply(fluctuate)

# diagonal symmetry
relation_df = relation_df + relation_df.T

product_df

,Unnamed: 0,sn,name,image_url,description,des_vec
0,0,MU01,雙面可穿鋪棉連帽大衣,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,男裝雙面可穿鋪棉連帽大衣NT 軍綠,0.080488365143537520.356602147221565252.267066...
1,1,MU02,雙面可穿鋪棉連帽大衣,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,男裝雙面可穿鋪棉連帽大衣NT 米白,0.212551441043615340.251976259052753452.238863...
2,2,MU03,棒球外套,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,男裝棒球外套STUDIO SEVEN 灰色,0.47258463315665720.33003669604659081.65472375...
3,3,MU04,棒球外套,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,男裝棒球外套STUDIO SEVEN 黑色,0.41967347078025340.446001973003149031.5940554...
4,4,MU05,寬版鋪綿牛仔外套,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,男裝寬版鋪綿牛仔外套,-0.035820128396153450.273835081607103351.27859...
...,...,...,...,...,...,...
97,97,FU14,多層次穿搭針織衫,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,女裝多層次穿搭針織衫(附背心兩件組)(長袖) 看起來像層搭圓弧衣襬襯衫的假兩件針織衫。在好搭...,1.49898556992411610.00395347760058939519.02054...
98,98,FU15,羅紋中高領針織衫,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,女裝羅紋中高領針織衫(長袖) 採用具時尚感的中高領設計，展現俐落感的羅紋針織衫。使用不易起毛...,0.36983149405568841.41896077804267415.46231861...
99,99,FU16,羅紋中高領針織衫,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,女裝羅紋中高領針織衫(長袖) 採用具時尚感的中高領設計，展現俐落感的羅紋針織衫。使用不易起毛...,0.25986526068300011.269314689561724715.3348042...
100,100,FU17,Soft RichV領針織長上衣,https://im.uniqlo.com/images/tw/gu/pc/goods/32...,女裝Soft RichV領針織長上衣(長袖) 可露出美麗肩頸的V領針織長上衣。特色是蓬鬆柔軟...,-0.7490366331767291-0.964204064570367324.99481...


In [ ]:
relation_df

,MU01,MU02,MU03,MU04,MU05,MU06,MU07,MU08,MU09,MU10,MU11,MU12,MU13,MU14,MU15,MU16,MU17,MU18,MU19,MU20,MU21,MU22,MU23,MU24,MU25,MU26,MU27,MU28,MU29,MU30,MF01,MF02,MF03,MF04,ML01,ML02,ML03,ML04,ML05,ML06,...,FF15,FF16,FF17,FF18,FF19,FF20,FL01,FL02,FL03,FL04,FL05,FL06,FL07,FL08,FL09,FL10,FL11,FL12,FL13,FL14,FL15,FL16,FU01,FU02,FU03,FU04,FU05,FU06,FU07,FU08,FU09,FU10,FU11,FU12,FU13,FU14,FU15,FU16,FU17,FU18
MU01,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.949,0.663,0.827,0.655,0.699,0.671,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MU02,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.719,0.815,0.676,0.842,0.837,0.671,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MU03,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.637,0.636,0.558,0.686,0.596,0.668,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MU04,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.532,0.210,0.285,0.515,0.520,0.558,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MU05,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.511,0.582,0.471,0.570,0.825,0.744,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FU14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.589,0.000,0.0,0.0,0.0,0.261,0.326,0.680,0.650,0.635,0.713,0.754,0.572,0.943,0.754,0.813,0.816,0.652,0.506,0.643,0.702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FU15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.557,0.586,0.0,0.0,0.0,0.837,0.764,0.757,0.748,0.766,0.731,0.662,0.842,0.666,0.671,0.824,0.832,0.213,0.248,0.772,0.604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
FU16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.000,0.631,0.0,0.0,0.0,0.791,0.750,0.798,0.739,0.795,0.794,0.816,0.824,0.804,0.762,0.665,0.686,0.433,0.278,0.567,0.181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
FU17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.624,0.602,0.0,0.0,0.0,0.591,0.559,0.813,0.831,0.832,0.849,0.693,

In [ ]:
"""Connecting to the sever"""
print('Connecting to the sever...')

# login info

sever_link = 'bolt://18.233.65.183:32968'
pws = 'narcotics-deletion-impedance'

# connect to Neo4j database
fashion_map = neo.Graph(sever_link, password=pws)

print('Done.')

Connecting to the sever...
Done.


In [ ]:
"""
Import Product Data to Nodes
Note: this script will overwrite your database.
"""

# clean the database
fashion_map.delete_all()

print('Connected successfully, and cleaned the database.')
print('Creating nodes...')


"""Create node"""
# container of nodes
num  = 5
cate_node = [None]*(num)

# gender
cate_node[0] = neo.Node('Gender', gender='Men', cht='男')
cate_node[1] = neo.Node('Gender', gender='Women', cht='女')

# form
cate_node[2] = neo.Node('Form', form='Full', cht='全身')
cate_node[3] = neo.Node('Form', form='Lower', cht='下著')
cate_node[4] = neo.Node('Form', form='Upper', cht='上衣')

# create
for n in range(num):
	fashion_map.create(cate_node[n])

# import product from csv
pgs = 0
product_node = []
for i in range(pn):
  product_node.append(neo.Node('Product', sn=product_df['sn'][i], 
								       name=product_df['name'][i],
		                   image_url=product_df['image_url'][i],
		                   description=product_df['description'][i],
                           vector=product_df['des_vec'][i]))
  fashion_map.create(product_node[i])
  # progress bar
  pgs += 1
  print('\r' + '[Creating]:[%s%s]%.2f%%' % ('█' * int(pgs*30/pn),
      ' ' * (30-int(pgs*30/pn)),
      float(pgs/pn*100)), end='')

print('\nDone.')

Connected successfully, and cleaned the database.
Creating nodes...
[Creating]:[██████████████████████████████]100.00%
Done.


In [ ]:
"""
Set Relationships
Note: node_generator script should be executed first.
"""

# create matcher
n_matcher = neo.NodeMatcher(fashion_map)

# get product node
product_node = n_matcher.match('Product')
product_node = list(product_node)
# print(product_node[0]['sn'][0])

print('Creating relationship...')

"""GENDER relationship"""

pgs = 0
for i in range(pn):
  if product_node[i]['sn'][0] == 'M':
    target_node = n_matcher.match('Gender', gender='Men') 
  else:
    target_node = n_matcher.match('Gender', gender='Women') 

  r = neo.Relationship(product_node[i], 'GENDER', list(target_node)[0])
  fashion_map.merge(r)

  # progress bar
  pgs += 1
  print('\r' + '[GENDER]:[%s%s]%.2f%%' % ('█' * int(pgs*30/pn),
      ' ' * (30-int(pgs*30/pn)),
      float(pgs/pn*100)), end='')
print()

"""FORM relationship"""

pgs = 0
for i in range(pn):
  if product_node[i]['sn'][1] == 'F':
    target_node = n_matcher.match('Form', form='Full') 

  elif product_node[i]['sn'][1] == 'L':
    target_node = n_matcher.match('Form', form='Lower')

  else:
    target_node = n_matcher.match('Form', form='Upper') 

  r = neo.Relationship(product_node[i], 'FORM', list(target_node)[0])
  fashion_map.merge(r)
 
  # progress bar
  pgs += 1
  print('\r' + '[FORM]:[%s%s]%.2f%%' % ('█' * int(pgs*30/pn),
      ' ' * (30-int(pgs*30/pn)),
      float(pgs/pn*100)), end='')

print('\nDone.')

Creating relationship...
[GENDER]:[██████████████████████████████]100.00%
[FORM]:[██████████████████████████████]100.00%
Done.


In [ ]:
"""SAME and FIT relationship"""

print('Creating relationship...')
pgs = 0

for i in range(pn):
  for j in range(pn):
    code = relation_df[psn[i]][psn[j]]
    if code <= 0.0:
      pass
    elif code == 2:
      r = neo.Relationship(product_node[i], 'SAME', product_node[j])
      fashion_map.merge(r)
    else:
      r = neo.Relationship(product_node[i], 'FIT', product_node[j])
      r['score'] = round(code, 3)
      fashion_map.merge(r)
   
  # progress bar
  pgs += 1
  print('\r' + '[SAME&FIT]:[%s%s]%.2f%%' % ('█' * int(pgs*30/pn),
     ' ' * (30-int(pgs*30/pn)),
     float(pgs/pn*100)), end='')
  
print('\nDone.')

Creating relationship...
[SAME&FIT]:[██████████████████████████████]100.00%
Done.
